In [2]:
import random

# Lit 🔥 Review Visualiser

### team notes
Below is a basic outline of our data structure, a directed graph

It's heavily commented (some of which won't make it to submission) to keep it as clear as possible

Shout if anything is confusing and we'll figure it out, or if I've misinterpreted any of the decisions we made the first time we talked about it 

## Data Structure

### note on classes
this is just for clarity, won't make the final report

We _could_ represent the graph without declaring a new class, but this will keep things cleaner for us in the long run

The alternative would probably to create a dictionary of lists, that would look a little like this

In [3]:
graph = {
    'adjacency_list': [[], [], []], # what nodes are connected to what nodes, every node would have it's own list
    'node': [1, 69, 420] # just a list of all the nodes
}

We would then have to define functions to add/get nodes and edges anyway, for example:

In [4]:
def add_node(graph, x):
    graph['adjacency_list'].append([])
    graph['nodles'].append(x)

And then to _get_ a node, it would pass the index, much like the method in the class

In [5]:
def get_node(graph, index):
    return graph['nodes'][index]

When you see *self* as one of the variables in the below class method, that just means we want to act on itself, so it saves us having to pass the `graph` variable to every function we all

## _Actual_ Data Structure

Shout if any of the syntax is confusing

In [6]:
class Graph:
    
    def __init__(self) -> None:
        """this function will be called when we create a new Graph;
        i.e. graph = Graph();
        it just initializes two empty lists,
        one for each node and one for for all the nodes it is connected to"""
        self.__adjacency_list: list = [] # self.whatever just means that the list is an attribute of the Graph
        self.__nodes: list = []

    def add_node(self, node) -> None:
        """this function accepts a node as an argument
        and add it to the nodes list, then adds an empty
        list to the adjacency_list that we will populate
        with the nodes it is connected to"""
        self.__adjacency_list.append([])
        self.__nodes.append(node)
        
    def add_edge(self, nodeX, nodeY) -> None:
        """this function accepts two nodes, arbitrarily X and Y;
        adding nodeY to the adjacency list of nodeX;
        NOTE: it won't add nodeX to the adjacency list of nodeY 
        because in our implementation the graph is directed:
        references only go one way"""
        if nodeY not in self.__adjacency_list[nodeY]:  # Ensure unique edges
            self.__adjacency_list[nodeX].append(nodeY)
        
    def neighbours(self, node):
        """this function just returns the adjacency_list 
        (all the nodes connected) to any given node as an argument"""
        return self.__my_adjacency_list[node]
        
    def __str__(self) -> str:
        """printing, mainly just debuggin, not exciting"""
        out: str = ""
        for i in range(len(self.__adjacency_list)):
            out.append("node", i, "(", self.__nodes[i], ") = ", self.__adjacency_list[i])
        return out

    def get_nodes(self) -> list:
        """this function just returns all the nodes"""
        return self.__nodes
    
    def get_adjacency_list(self) -> list[list]:
        """this function just returns the adjacency list for all the nodes"""
        return self.__adjacency_list

This graph doesn't quite serve our purposes just yet

From what I remember of what we decided, we wanted to be able to add one node (a paper) to the graph, then for every paper it referenced, at it to the graph

The for each of the new nodes added, add their references (nodes) to the graph as well

And then add _their_ references to the graph, until we get to some arbitrary distance (number of connections) from the original paper (node) than we added

So the paper will contain some information: it will know what papers it references, but we also need a way to assign it a distance from the original paper

So let's define a Paper class

In [7]:
class Paper:

    def __init__(self, references):
        """constructor to create a Paper,
        it needs to know what it's references are
        and it's distance from the original paper:
        it's default will be 0 (each paper assumes it is the 
        original paper)"""
        self.__refs = references
        self.__dist = 0

    def get_refs(self):
        return self.__refs
    
    def get_dist(self):
        return self.__dist
    
    def set_dist(self, distance):
        self.__dist = distance

# THE PAPER IS THE GRAPH

what does that mean? Welllll a Paper is an inherenetly recursive object in the real world, it has references which are papers which have references which are papers... 

So the object that we create to represent a Paper needs to represent this property, however we need to add a base case (a depth at which we no longer care about the references)

The real world has no recursion depth limits but python does

So let's try again and create a different Paper class

In [79]:
class Paper:

    def __init__(self, name = 0, distance = 0):
        self.__name = name
        self.__distance = distance
        self.__references = self.create_references()

    def create_references(self):
        # return an empty list if we have exceeded relevant depth
        if self.get_distance() > 3:
            return []
        # otherwise, return a list of Paper objects
        return [Paper(random.randint(1, 10) * n, self.get_distance() + 1) for n in range(1, 6)]

    def get_name(self):
        return self.__name
    
    def get_distance(self):
        return self.__distance

    def find_distance(self, name = 0, distance = 0):
        if name == self.get_name():
            return self.get_distance()
        else:
            for paper in self.get_references():
                dist = paper.find_distance(name, distance + 1)
                if dist is not None:
                    return dist
            return None


    def get_references(self):
        return self.__references

    def __repr__(self):
        return f"{self.get_name()}: {self.get_references()}"

In [80]:
paper = Paper()

In [92]:
paper.get_references()

[8: [7: [8: [7: [], 6: [], 12: [], 28: [], 35: []], 2: [2: [], 8: [], 9: [], 28: [], 10: []], 15: [9: [], 16: [], 3: [], 8: [], 10: []], 28: [4: [], 8: [], 24: [], 16: [], 15: []], 5: [1: [], 12: [], 21: [], 32: [], 10: []]], 4: [3: [3: [], 18: [], 3: [], 32: [], 25: []], 20: [8: [], 20: [], 15: [], 40: [], 20: []], 27: [10: [], 10: [], 27: [], 16: [], 20: []], 12: [6: [], 6: [], 15: [], 20: [], 5: []], 25: [3: [], 20: [], 6: [], 4: [], 50: []]], 27: [7: [7: [], 2: [], 27: [], 8: [], 30: []], 4: [8: [], 8: [], 18: [], 24: [], 5: []], 9: [8: [], 4: [], 30: [], 4: [], 30: []], 36: [3: [], 18: [], 3: [], 40: [], 20: []], 10: [7: [], 20: [], 27: [], 32: [], 45: []]], 16: [4: [10: [], 20: [], 6: [], 20: [], 35: []], 14: [10: [], 6: [], 30: [], 28: [], 50: []], 9: [9: [], 4: [], 21: [], 32: [], 20: []], 12: [8: [], 8: [], 15: [], 4: [], 15: []], 25: [3: [], 8: [], 27: [], 4: [], 40: []]], 15: [9: [8: [], 2: [], 30: [], 4: [], 45: []], 10: [4: [], 4: [], 15: [], 32: [], 50: []], 27: [6: [], 8

In [94]:
paper.find_distance(18)

4

In [90]:
paper.find_distance(2)

3